In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2026-02-05 21:49:43--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  97.8MB/s    in 0.3s    

2026-02-05 21:49:43 (97.8 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",             ## character encoding. ..why not UTF-8 ??. ..
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
df_ratings.tail()

,user,isbn,rating
1149775,276704,1563526298,9.0
1149776,276706,0679447156,0.0
1149777,276709,0515107662,10.0
1149778,276721,0590442449,10.0
1149779,276723,05162443314,8.0


In [6]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   isbn    271379 non-null  object
 1   title   271379 non-null  object
 2   author  271377 non-null  object
dtypes: object(3)
memory usage: 6.2+ MB


In [7]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   user    1149780 non-null  int32  
 1   isbn    1149780 non-null  object 
 2   rating  1149780 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 17.5+ MB


In [8]:
len(df_ratings.user.unique())

105283

In [9]:
## clean: remove from the dataset users with less than 200 ratings and books with less than 100 ratings

df = df_ratings

k1 = df['user'].value_counts()   ## unique users - how many times each user rated any book
k2 = df['isbn'].value_counts()   ## unique books - how many times a book (isbn) was rated

df = df[~df['user'].isin(k1[k1 < 200].index)]   ## remove columns with users who appear less than 200 times (i.e. gave less than 200 ratings)
df = df[~df['isbn'].isin(k2[k2 < 100 ].index)]  ## and books that appear less than 100 times (i.e. was rated less than 100 times)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49781 entries, 1456 to 1147441
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    49781 non-null  int32  
 1   isbn    49781 non-null  object 
 2   rating  49781 non-null  float32
dtypes: float32(1), int32(1), object(1)
memory usage: 1.1+ MB


In [10]:
df = pd.merge(right=df, left=df_books, on='isbn')  ## combining the ratings and books df's as they both have an 'isbn' column. ..
df = df.drop_duplicates(['title','user'])
df.head()

,isbn,title,author,user,rating
0,0440234743,The Testament,John Grisham,277478,0.0
1,0440234743,The Testament,John Grisham,2977,0.0
2,0440234743,The Testament,John Grisham,3363,0.0
3,0440234743,The Testament,John Grisham,7346,9.0
4,0440234743,The Testament,John Grisham,9856,0.0


In [11]:
## only interested in the book title, user, and rating. ..so reshape df
## for a book a user didn't rate fill 0

df = df.pivot(index='title', columns='user', values='rating').fillna(0)
df.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
## matrix to fit in KNN model. ..

matrix = df.values
matrix

array([[ 9.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 10.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [13]:
## metric - how to determine distance between data points (choice of parameter depedns on type of dataset)
## algorithm - how to determine neighbours in data set. ..brute: simplest approach, suitable for small-moderate size datasets

KNN_model = NearestNeighbors(metric='cosine', algorithm='brute')
KNN_model.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [14]:
k = df.loc['4 Blondes'].array.reshape(1,-1)

d, i = KNN_model.kneighbors(k,n_neighbors=6)
y = zip(d[0], i[0])
list(y)

[(np.float64(2.220446049250313e-16), np.int64(3)),
 (np.float64(0.7986316484713514), np.int64(519)),
 (np.float64(0.8302026571802759), np.int64(385)),
 (np.float64(0.8377817409877164), np.int64(473)),
 (np.float64(0.8379611544982923), np.int64(93)),
 (np.float64(0.8428564490939956), np.int64(330))]

In [15]:
# function to return recommended books - this will be tested

## function to recommend books (recommend books/datapoints in the same "neighborhood")

def get_recommends(book = ""):
  x = df.loc[book].array.reshape(1,-1)     ## 1D array representing a datapoint
  dist, n_index = KNN_model.kneighbors(x, n_neighbors=6)
  recs = []
  for d, i in zip(dist[0], n_index[0]):
    if d != 0:
      n_book = df.index[i]
      recs.append([n_book, d])
  recommended_books = [book, recs[::-1]]     ## give back a list of the ref book, and the neighbors/recommends in descending order of distances

  return recommended_books

In [16]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", np.float64(0.8016210581447822)],
  ['The Weight of Water', np.float64(0.7708583572697412)],
  ['The Surgeon', np.float64(0.7699410973804288)],
  ['I Know This Much Is True', np.float64(0.7677075092617776)],
  ['The Lovely Bones: A Novel', np.float64(0.7234864549790632)],
  ["Where the Heart Is (Oprah's Book Club (Paperback))",
   np.float64(2.220446049250313e-16)]]]

In [17]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", np.float64(0.8016210581447822)], ['The Weight of Water', np.float64(0.7708583572697412)], ['The Surgeon', np.float64(0.7699410973804288)], ['I Know This Much Is True', np.float64(0.7677075092617776)], ['The Lovely Bones: A Novel', np.float64(0.7234864549790632)], ["Where the Heart Is (Oprah's Book Club (Paperback))", np.float64(2.220446049250313e-16)]]]
You passed the challenge! 🎉🎉🎉🎉🎉
